# Fairness, Accountability, Transparency and Ethics Course (FATE)

## Universitat Pompeu Fabra (UPF)
### Year 23/24
### Author: Manuel Portela (manuel.portela@upf.edu) with the contribution from Ioannis Bilionis (ibilionis@adherahealth.com)
*** Partially based on the original exercises made by David Solans (david.solans@upf.edu) ***
<br>
<br>
<br>
Submission date: 22/03/2024 at 23:59 on Aula Global

Please, implement this notebook **in pairs**.

<br>
<br>

**Legend** <br>
In this notebook we use:    
<div class="alert alert-block col-md-7 alert-info">To recall information from the theory classes and other tips</div>
<div class="alert alert-block col-md-7 alert-warning">To point important things that should not be 
   forgotten</div> 
<div class="alert alert-block col-md-7 alert-primary bg-primary">LAB TASK</div>

# 3. Part 2: Predicting with a COVID-19 Dataset

This project consist on applying the concepts and the techniques acquired during the course, with the addition of mitigation algorithms. 

The purpose of this project is to train a model that answer a specific question. Each group will have to choose a question and will have a subset of the original dataset to use. 

<div class="alert alert-block alert-info">We will work with subsets obtainef from the full dataset available at <a href="https://www.kaggle.com/datasets/omarlarasa/cov19-open-data-mexico">kaggle</a>.</div>

Use the question that correlates to your group with its corresponding dataset:
1. What is the probability of dying increase with the patient's age? 
2. What does probability change if the patient has another health issue? 
3. How dying probability increases when the patient has been intubated? 
4. Is it true that smoking increases the probability of dying by COV19?
5. Do pregnant women has more probability of being intubated?
6. How is the probability of hospitalization and probability of death are related to the length of symptoms?  
7. Is there a potential discrimination against Migrants? Hint: calculate the ratio between death and ICU/intubation


<div class="alert alert-block alert-success">
    <p><b>[To complete]</b></p>
    <p>Group number: 6</p>
    <p>Group members: Aitana Gonzalez, Berta Mitjavila </p>
</div>

###  Libraries used in this notebook
You will need to install: **numpy**, **pandas**, **matplotlib**, and **sklearn**.

In [2]:
## Required statements
import pandas as pd
import numpy as np

from sklearn import linear_model
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score


from aif360.datasets import BinaryLabelDataset
from aif360.datasets import AdultDataset
from aif360.metrics import BinaryLabelDatasetMetric
from aif360.metrics import ClassificationMetric

from aif360.algorithms.preprocessing.optim_preproc_helpers.data_preproc_functions\
        import load_preproc_data_adult

from aif360.metrics import BinaryLabelDatasetMetric

from IPython.display import Markdown, display


from IPython.display import Markdown, display
import matplotlib.pyplot as plt
%matplotlib inline

pip install 'aif360[LawSchoolGPA]'
pip install 'aif360[AdversarialDebiasing]'
pip install 'aif360[AdversarialDebiasing]'
pip install 'aif360[Reductions]'
pip install 'aif360[Reductions]'
pip install 'aif360[Reductions]'


<div class="alert alert-block alert-primary bg-primary head-2">
    <h2 class="alert-heading">[L3.1] Load and analyse your dataset (1 point)</h2>
    <hr>
     <p class="mb-0">Load your assigned dataset and answer the following questions:</p>

   <ol>
    <li>Is it k-anonymous?  [Interpret]</li>
    <li>Does it complies with l-diversity ? [interpret]</li>
    <li>Is it balanced? [Interpret]</li>
    <li>Select your protected class [justify]</li>
    <li>Define sensitive attributes [justify]</li>
   </ol>
</div>

In [11]:
df = pd.read_csv("../Data/decease_dataset_v3.csv")
df.head()

,respiratory_hospital_unit,sex,admission_date,symptoms_onset_date,decease_date,intubation,pneumonia,age,native_speaker,native_mexican,...,immunosuppression,hypertension,other_diseases,cardiovascular,obesity,chronic_renal_failure,smoker,ICU_admission,covid_positive,deceased_patient
0,1.0,1.0,04/30/2020,04/28/2020,NaN,0.0,1.0,NaN,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0
1,0.0,2.0,05/14/2020,05/11/2020,NaN,NaN,0.0,61.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,1,0
2,0.0,1.0,05/06/2020,05/06/2020,NaN,NaN,0.0,44.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,NaN,0,0
3,0.0,2.0,05/16/2020,05/15/2020,NaN,NaN,0.0,41.0,0.0,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,NaN,0,0
4,1.0,2.0,04/08/2020,04/07/2020,NaN,0.0,1.0,79.0,0.0,0.0,...,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0,0


In [12]:
df.columns.to_list()

['respiratory_hospital_unit',
 'sex',
 'admission_date',
 'symptoms_onset_date',
 'decease_date',
 'intubation',
 'pneumonia',
 'age',
 'native_speaker',
 'native_mexican',
 'diabetes',
 'epoc',
 'asma',
 'immunosuppression',
 'hypertension',
 'other_diseases',
 'cardiovascular',
 'obesity',
 'chronic_renal_failure',
 'smoker',
 'ICU_admission',
 'covid_positive',
 'deceased_patient']

In [13]:
# We will work only with covid_positive = 1 because we want the probability of dying due to covid
df = df[df['covid_positive'] == 1]

# As covid_positive is always 1, it is unuseful
# Attributes like 'admission_date' are not affected by the age so are irrelevant
df = df.drop(columns=['covid_positive', 'admission_date', 'symptoms_onset_date', 'decease_date'])

df.head()

,respiratory_hospital_unit,sex,intubation,pneumonia,age,native_speaker,native_mexican,diabetes,epoc,asma,immunosuppression,hypertension,other_diseases,cardiovascular,obesity,chronic_renal_failure,smoker,ICU_admission,deceased_patient
1,0.0,2.0,NaN,0.0,61.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0
6,0.0,2.0,NaN,0.0,47.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0
7,1.0,1.0,NaN,0.0,44.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,NaN,0
10,1.0,1.0,0.0,0.0,78.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1
14,1.0,2.0,0.0,1.0,68.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1


### K-anonimity

**Identifiers:** These are unique attributes that can be used to identify an individual. In this dataset, there doesn’t seem to be any explicit identifiers like name, social security number, or patient ID.

**Quasi-identifiers:** These are attributes that, in combination, can be used to identify an individual. In this dataset, the following could be considered quasi-identifiers: ‘sex’, ‘age’, ‘native_speaker’, ‘native_mexican’.

**Confidential attributes:** These are sensitive attributes that should be protected. In this dataset, the following could be considered confidential attributes: ‘respiratory_hospital_unit’, ‘intubation’, ‘pneumonia’, ‘diabetes’, ‘epoc’, ‘asma’, ‘immunosuppression’, ‘hypertension’, ‘other_diseases’, ‘cardiovascular’, ‘obesity’, ‘chronic_renal_failure’, ‘smoker’, ‘ICU_admission’, ‘deceased_patient’.

In [14]:
## Use a helper function that calculates the groups by value
## If using pandas, you can consider using grouping and group sizes for that
def get_values_frequencies(_df, quasi_identifier_cols):
    return _df.groupby(quasi_identifier_cols).size().reset_index(name='Counts')

## Code a function that uses the function above and determines whether a given dataset satifies K-anonimity 
# for a given K
def is_k_anonymous(_df, K, quasi_identifier_cols):
    value_counts = get_values_frequencies(_df, quasi_identifier_cols)
    max_k = min(value_counts['Counts'])
    if K < max_k:
        return str(all(value_counts['Counts'] >= K)) + f", but it is {max_k}-anonymous too." 
    else:
        return all(value_counts['Counts'] >= K)

In [18]:
print("Given the previously established quasi-identifiers, does the dataset satisfy 2-anonymity)?", is_k_anonymous(df, 2, ["sex", "age", "native_speaker", "native_mexican"]))


Given the previously established quasi-identifiers, does the dataset satisfy 2-anonymity)? False


As we can observe, with the given quasi-identifiers, the dataset is not 2-anonymous. If a dataset does not satisfy 2-anonymity, it means that there are individuals in the dataset who can be uniquely identified based on the quasi-identifiers.

In [19]:
df.groupby(by=["sex", "age", "native_speaker", "native_mexican"]).size()

sex  age   native_speaker  native_mexican
1.0  12.0  0.0             0.0                 4
     13.0  0.0             0.0                 3
     14.0  0.0             0.0                 4
     15.0  0.0             0.0                11
     16.0  0.0             0.0                 5
                                            ... 
2.0  88.0  0.0             0.0               140
           1.0             0.0                 2
     89.0  0.0             0.0               143
           1.0             1.0                 1
     90.0  0.0             0.0               104
Length: 416, dtype: int64

The previous output shows the count of records for each unique combination of the four quasi-identifiers, and we can observe that there is an individual that can be uniquely identified.

In [20]:
df[(df['sex'] == 2.0) & (df['age'] == 89.0) & (df['native_speaker'] == 1.0) & (df['native_mexican'] == 1.0)]

,respiratory_hospital_unit,sex,intubation,pneumonia,age,native_speaker,native_mexican,diabetes,epoc,asma,immunosuppression,hypertension,other_diseases,cardiovascular,obesity,chronic_renal_failure,smoker,ICU_admission,deceased_patient
17895,1.0,2.0,0.0,1.0,89.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1


Given this situation, we will have to **anonymize** the dataset. In this case, we could apply generalization to the ‘age’ attribute.

<div class="alert alert-block alert-primary bg-primary head-2">
    <h2 class="alert-heading">[L3.2] Prepare your dataset and train a model (0.5 point)</h2>
    <hr>
    <p class="mb-0">Split your dataset for train and test [Code]</p>
     <p>Then, train a classifier (scikit-learn model) [Code] considering the question that you should solve [justify]. </p>
</div>



In [1]:
## Your code here

<div class="alert alert-block alert-primary bg-primary head-2">
    <h2 class="alert-heading">[L3.3] Assess your model performance (1 point)</h2>
    <hr>
    <p class="mb-0">You should analyse the following analysis:</p>
    <ol>
        <li>Performance metrics [interpret]</li>
        <li>Use Specificity to understand how well it classifies TRUE NEGATIVES (important for COVID) [Interpret</li>
        <li>Training a XGB regressor analyse the risk factors used in this dataset and how they impact on your protected class.  [Code and Interpret]</li>
    </ol>
     
</div>

<div class="alert alert-block alert-info col-md-12">
    <p><b>Specificity metric:</b> (true negative rate) is the probability of a negative test result, conditioned on the individual truly being negative. This is important on COVID-19 detection because we know that is hard to detect the infection, therefore there has been many false negatives, resulting in underidentification.</p>
    <p><b>Analysing factors:</b> When using multiple attributes for a classifier, each attribute can have a different impact in the precision of the algorithm, which also varies regarding the dataset. Understanding the weights of each factor can help to understand what are the effects on the protected class.</p>
</div>

In [30]:
## Your code here

<div class="alert alert-block alert-primary bg-primary head-2">
    <h2 class="alert-heading">[L3.4] Assess the fairness metrics (1 point)</h2>
    <hr>
    <p class="mb-0">Assess the performance from the fairness point of view of the created model.</p>
    <p>For that, you should plot the result of some fairness metric for classification.</p>
    <p> You can just use the same metrics we used in Part 1, but remember to add other metrics if you need them to detect changes of performance in the following exercises.</p>
</div>

In [ ]:
## Your code here

### Unfairness mitigation algorithms  in AIF360
The library contains a set of algorithms to mitigate potential unfairness in machine-learning pipelines.

The provided algorithms correspond implementation of to state of the art techniques (most of them contain a link to a scientific paper that introduced them).

The existing algorithms can be found [here](https://aif360.readthedocs.io/en/latest/modules/algorithms.html).

They are divided between:
- **Preprocessing.** Algorithms that are applied before starting the learning process (modification of the input data before feeding it to the model).

- **Inprocessing.** Algorithms that are applied during the learning process (e.g.: modification of the loss function to account for fairness).

- **Postprocessing.** Algorithms that are applied after the predictions of the models, substituting them for more fair solutions.

<div class="alert alert-block alert-primary bg-primary head-2">
    <h2 class="alert-heading">[L3.5] Apply mitigation algorithms to your model (0.5 points)</h2>
    <hr>
     <p class="mb-0">Apply an unfairness mitigation algorithm of your election, train a new classifier of the same family used above and assess the new performance from the fairness point of view.</p>
    <p>Provide a textial description of the results (max 3 lines).</p>
</div>



<div class="alert alert-block alert-info col-md-12">

<p>Although you can try other algorihtms if you want, the suggested algorithms to mitigate unfairness are the following:</p>

    
<ul>
<li><b><a href="https://aif360.readthedocs.io/en/latest/modules/generated/aif360.algorithms.preprocessing.DisparateImpactRemover.html" >Preprocessing</a></b>
<ul>
<li>Disparate Impact Remover</li>
<li>Rewighing (you need a classifier that allows you to parametrize sample weights for this)</li>
</ul>

</li>
<li><b><a href="https://aif360.readthedocs.io/en/latest/modules/algorithms.html#aif360-algorithms-inprocessing">Inprocessing</a></b>
<ul>
<li>MetaFairClassifier</li>
</ul>
</li>
<li><b><a href="https://aif360.readthedocs.io/en/latest/modules/algorithms.html#module-aif360.algorithms.postprocessing">Postprocessing</a></b>
<ul>
<li>Disparate Impact Remover</li>
<li>EqOddsPostprocessing (You need to use the equalized odds ratio to measure performance of this classifier and the classifier trained above)</li>
</ul>
</li>
</ul>

<p>You can add necessary imports for using the algorithm</p>

</div>


<div class="alert alert-block alert-warning col-md-12">
    <p>Sometimes, certain algorithms do not work well with certain classifiers / output distributions</p>
    <p>This can happen for several reasons, one of them being the changes done for subsequent versions of the tool</p>
    <br>
    <p><b>If you see that your unfairness mitigation algorithm does not work as expected, do not invest too much time trying to solve it, use another algorithm.</b></p>
</div>   

In [ ]:
## Your code here

<div class="alert alert-block alert-primary bg-primary head-2">
    <h2 class="alert-heading">[L3.6] Assess the effects of the mitigation technique (1 point)</h2>
    <hr>
     <p class="mb-0">Analyse the performance of the effects of the mitigation algorithm in the performance of your model. You can iterate over your training to get the best result (in relation to you goal). Explain how do arrive to the best choice and why it is the best one.</p>
</div>


In [ ]:
## Your code here

<div class="alert alert-block alert-primary bg-primary head-2">
    <h2 class="alert-heading">[L3.7] Executive report (5 points)</h2>
    <hr>
     <p class="mb-0">After analysing your dataset and training your models, please create a 1-3 page report containing the following items (on your consideration): </p>
    <ul>
        <li>Anonymization of the dataset</li>
        <li>Fairness in original dataset</li>
        <li>Factors that impact on discrimination</li>
        <li>Fairness metrics</li>
        <li>Algorithms used for training the model</li>
        <li>Algorithmic performance expected and obtained</li>
        <li>Algorithmic metrics applied</li>
        <li>Algorithmic performance after the metrics</li>
        <li>Conclusions regarding the goal of your model and the expected utility in a real-case scenario</li>
    </ul>
</div>

<div class="alert alert-block alert-warning">
<p>You can include figure if you consider necessary, and you can reuse observations from the notebook, please avoid placing code in the report.</p>
<p>Please, include the contribution of each of the team mates to this project. </p>
</div> 
